# [SELECT PANDAS ROWS BASED ON CONDITION](https://chrisalbon.com/python/data_wrangling/pandas_selecting_rows_on_conditions/)

In [1]:
import os, sys, time
from time import sleep
import pathlib, fnmatch
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [2]:
from sqlalchemy import create_engine

---

## create `engine` from SQL-ALCHEMY

In [3]:
# CREATE SQL ALCHEMY ENGINE
engine = create_engine('sqlite:///e:/_Quotes_Tracker/data/quotes_tracker.db')

In [4]:
# CREATE CONNECTION TO DATABASE (from ENGINE)
conn = engine.connect()

---

# CREATE `PANDAS` DF FROM `SQL QUERY`

In [5]:
tracking_num_test = "T00000000003"

In [6]:
the_query = "SELECT * FROM [quotes] WHERE [tracking_number] = '" + str(tracking_num_test) + "'"

In [7]:
print(the_query)

SELECT * FROM [quotes] WHERE [tracking_number] = 'T00000000003'


In [8]:
# SQL_TABLE?
# [2020-01-21]\\SQL_Table = pd.read_sql_table(table_name="quotes", con=conn, index_col=["tracking_number"])
SQL_Table = pd.read_sql_query(sql=str(the_query), 
                              con=conn
                             )

In [9]:
SQL_Table  #["close_time"]["close_time"]["turn_around"]

,tracking_number,name,email,type,sent,open_time,close_time,turn_around,notes,initials,account_id,prodflow_quote_number,sap_quote_number,price,company_name,product_number
0,T00000000003,Raffaella,ultra@ultrasci.it,email,True,2020-01-14 11:31:59.582988,2020-01-14 11:55:00.000000,None,None,clr/SI,70599064,011420-790,3119120,None,ULTRA Italia,CUS-00002440


In [10]:
is_sent = SQL_Table["turn_around"].isin(["None"])

In [11]:
is_sent

0    True
Name: turn_around, dtype: bool

In [12]:
if bool(is_sent[0]) is False: #{
    print("YES TURN AROUND!")
    print(SQL_Table["turn_around"])
#}
else: # {
    print("NO TURN AROUND!")
    print(SQL_Table["close_time"])
#}

NO TURN AROUND!
0    2020-01-14 11:55:00.000000
Name: close_time, dtype: object


In [16]:
tracking_num_test_2 = "T000000000002"

In [17]:
the_query_2 = "SELECT * FORM [quotes] WHERE [tracking_number] = '" + str(tracking_num_test_2) + "'"

In [18]:
print(the_query_2)

SELECT * FORM [quotes] WHERE [tracking_number] = 'T000000000002'


In [19]:
SQL_Table_2 = pd.read_sql_query(sql=str(the_query),
                               con=conn
                               )

In [20]:
SQL_Table_2

,tracking_number,name,email,type,sent,open_time,close_time,turn_around,notes,initials,account_id,prodflow_quote_number,sap_quote_number,price,company_name,product_number
0,T00000000003,Raffaella,ultra@ultrasci.it,email,True,2020-01-14 11:31:59.582988,2020-01-14 11:55:00.000000,None,None,clr/SI,70599064,011420-790,3119120,None,ULTRA Italia,CUS-00002440


### Method 1: Using Boolean Values

In [8]:
# create variable with TRUE IF sent is TRUE
all_sent = SQL_Table['sent'] == "True"

In [9]:
# create variable with TRUE if turn_around is NOT "None"
already_sent = SQL_Table['turn_around'] != "None"

In [10]:
# create variable with TRUE if turn_around is "None"
recently_sent = SQL_Table["turn_around"] == "None"

In [11]:
# select all cases where SENT IS TRUE and TURN_AROUND is NONE
need_timestamps = SQL_Table[all_sent & recently_sent]

In [12]:
need_timestamps

,name,email,type,sent,open_time,close_time,turn_around,notes,initials,account_id,prodflow_quote_number,sap_quote_number,price,company_name,product_number
tracking_number,,,,,,,,,,,,,,,
T00000000003,Raffaella,ultra@ultrasci.it,email,True,2020-01-14 11:31:59.582988,2020-01-14 11:55:00.000000,None,None,clr/SI,70599064,011420-790,3119120,None,ULTRA Italia,CUS-00002440
T00000000004,Glen Arrieta,garrieta@ocwd.com,email,True,2020-01-14 11:46:15.654761,2020-01-14 12:48:00.000000,None,None,clr/SI,70339639,011420-791,3119191,None,Orange County Water District,CUS-00002442


In [13]:
type(need_timestamps)

pandas.core.frame.DataFrame

In [14]:
sent_df = SQL_Table[all_sent]

In [15]:
sent_df

,name,email,type,sent,open_time,close_time,turn_around,notes,initials,account_id,prodflow_quote_number,sap_quote_number,price,company_name,product_number
tracking_number,,,,,,,,,,,,,,,
T00000000001,Raffaella,ultra@ultrasci.it,email,True,2020-01-14 09:06:26.827094,2020-01-14 09:52:12.998358,0 days 00:45:46.171264,None,clr/cb,70599064,011420-787,3118922,None,ULTRA Italia,CUS-00002438
T00000000002,Rune Jorgenson,rmj@analytech.dk,email,True,2020-01-14 11:30:10.220050,2020-01-14 15:54:31.648525,0 days 04:24:21.428475,*Quote to be sent to Agilent rep. Susanne K.\n...,RF/cb/KSR,70209452,011420-793,3119213,None,Analytech,CUS-00002441
T00000000003,Raffaella,ultra@ultrasci.it,email,True,2020-01-14 11:31:59.582988,2020-01-14 11:55:00.000000,None,None,clr/SI,70599064,011420-790,3119120,None,ULTRA Italia,CUS-00002440
T00000000004,Glen Arrieta,garrieta@ocwd.com,email,True,2020-01-14 11:46:15.654761,2020-01-14 12:48:00.000000,None,None,clr/SI,70339639,011420-791,3119191,None,Orange County Water District,CUS-00002442
T00000000005,Raffaella,ultra@ultrasci.it,email,True,2020-01-14 12:03:37.168555,2020-01-16 11:39:33.472533,1 days 23:35:56.303978,None\n\n,clr/SI,70599064,011420-792,3119216,None,ULTRA Italia,CUS-00002443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T00000000080,Raffaella,ultra@ultrasci.it,email,True,2020-01-21 10:13:57.005142,2020-01-21 11:25:06.744189,0 days 01:11:09.739047,None\n,clr/SI,70599064,012120-822,3126691,None,ULTRA Italia,ICUS-0000984
T00000000081,Raffaella,ultra@ultrasci.it,email,True,2020-01-21 10:14:27.391875,2020-01-21 11:25:32.031611,0 days 01:11:04.639736,None\n,clr/SI,70599064,012120-823,3126712,None,ULTRA Italia,ICUS-0000985
T00000000085,Raffaella,ultra@ultrasci.it,email,True,2020-01-21 10:16:48.359267,2020-01-21 12:52:02.651244,0 days 02:35:14.291977,None\n,clr/SI,70599064,012120-831,3127112,None,ULTRA Italia,ICUS-0000986


In [16]:
the_tracking_number_str = "T00000000065"

In [32]:
# CLOSING TIME~
# sent_df.loc[['T00000000060', 'T00000000065']]
closing_time = sent_df['close_time'].loc[the_tracking_number_str]
print(closing_time)

2020-01-21 08:37:30.956994


### *if `turn_around` has not been computed yet...*

Loop thru...

Calculate `turn_around` time...

call `execute_db_query` **ONLY** to update *those* records

In [35]:
# counter
x = 0
while x < len(need_timestamps): # {
    # print(need_timestamps.iloc[x, 0])
    print("tracking_number == " + str(need_timestamps.index[x]))
    # DETERMINE ORIGINAL CLOSING TIME
    og_closing_time = sent_df['close_time'].loc[str(need_timestamps.index[x])]
    print("closing_time == " + str(og_closing_time) + "\n")
    x += 1
#}

tracking_number == T00000000003
closing_time == 2020-01-14 11:55:00.000000

tracking_number == T00000000004
closing_time == 2020-01-14 12:48:00.000000



---

## OLD IS BELOW

In [24]:
# counter 
x = 0
# WHILE WE ARE STILL LOOPING THRU DATAFRAME...
while x < len(need_timestamps): # {
    print("===========")
    tracking_number = str(need_timestamps.iloc[x, 0])
    name = str(need_timestamps.iloc[x, 1])
    email = str(need_timestamps.iloc[x, 2])
    type_ = str(need_timestamps.iloc[x, 3])
    sent = str(need_timestamps.iloc[x, 4])
    open_time = str(need_timestamps.iloc[x, 5])
    close_time = str(need_timestamps.iloc[x, 6])
    turn_around = str(need_timestamps.iloc[x, 7])
    print(tracking_number, sent, open_time, close_time, turn_around)
    ######################################
    # create timestamp for meow
    meow_ts = pd.Timestamp.now()
    # COMPUTE TURN AROUND TIME
    time_start = pd.Timestamp(open_time)
    print("TIME START == " + str(time_start)[:19])
    print("TIME END == " + str(meow_ts)[:19])
    run_time = str(meow_ts - time_start)[:18]
    print("RUN TIME == " + str(run_time))
    query = 'UPDATE quotes SET turn_around=? WHERE tracking_number=?'
    parameters = (str(run_time), tracking_number)
    ##############################################
    # INCREMENT COUNTER
    x += 1
    print("===========\n")
# }

T00000000003 True 2020-01-14 11:31:59.582988 2020-01-14 11:55:00.000000 None
TIME START == 2020-01-14 11:31:59
TIME END == 2020-01-21 10:51:11
RUN TIME == 6 days 23:19:12.29

T00000000004 True 2020-01-14 11:46:15.654761 2020-01-14 12:48:00.000000 None
TIME START == 2020-01-14 11:46:15
TIME END == 2020-01-21 10:51:11
RUN TIME == 6 days 23:04:56.22



In [ ]:
test_str = 'update QUOTES' \
           'SET name=' + str(need_timestamps.iloc())

In [ ]:
# FAKE QUERY AND SUBMIT?
query = 'update QUOTES' \
        'SET name=?, email=?, '

In [25]:
# TRY THE FOLLOWING
try: # {
    length = len(SQL_Table)
    print("LEN == " + str(length))
    # CREATING BOOL SERIES FROM isin()
    all_sent = SQL_Table["sent"].isin(["True"])
    print(len(all_sent))
    # SETTING DATA with "sent" = True < ONLY >
    recently_sent = SQL_Table["turn_around"].isin(["None"])
    print(len(recently_sent))
    # CREATE **ANOTHER** BOOL SERIES
    # (ones without any TURN_AROUND)
    newly_sent = all_sent["turn_around"].isin(["None"])
    print(len(newly_sent))
    # }
# }
except: # {
    errorMessage = str(sys.exc_info()[0]) + "\n"
    errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n"
    errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    typeE = str("TYPE : " + str(exc_type))
    fileE = str("FILE : " + str(fname))
    lineE = str("LINE : " + str(exc_tb.tb_lineno))
    messageE = str("MESG : " + "\n" + str(errorMessage))
    print("\n" + typeE +
                  "\n" + fileE +
                  "\n" + lineE +
                  "\n" + messageE)
# }
else: # {
    print("operation completed successfully..")
# }

LEN == 62
62
48
48
operation completed successfully..


In [ ]:
if __name__ == "__main__": # {
    print(len(SQL_Table))
# }